# Montar uma figura com 3 diagramas -> A - diagrama do sorotipo Ia, B - diagrama do sorotipo III, C - diagrama dos dois sorotipos Ia e III
## Esses diagramas de veen são utlizados para mostrar os sorotipos em comum dos 3 hospedeiros (Peixes, Humanos e Bovinos) estudados

In [ ]:
# Criar diretorio para os sorotipos
mkdir serotypeIa 
mkdir serotypeIII

In [ ]:
# Script para a contagem dos ISs do arquivo .fa

nano # Primeiramente vamos abrir o editor de texto nano para criar o script

# Apos abrir o editor, vamos escrever o seguinte código: copiar e colar o script abaixo

#!/usr/bin/env python3
"""
Extrator e Contador de Elementos IS (Insertion Sequences) de arquivos FASTA
Autor: Assistente Claude
Uso: python3 is_extractor.py <arquivo_fasta> [arquivo_saida]
"""

import sys
import re
from collections import defaultdict, Counter
import argparse

def parse_fasta(filename):
    """
    Parse um arquivo FASTA e retorna um dicionário com sequências
    """
    sequences = {}
    current_header = None
    current_sequence = []
    
    try:
        with open(filename, 'r') as file:
            for line in file:
                line = line.strip()
                if line.startswith('>'):
                    # Se já temos uma sequência anterior, salva ela
                    if current_header:
                        sequences[current_header] = ''.join(current_sequence)
                    
                    current_header = line[1:]  # Remove o '>'
                    current_sequence = []
                else:
                    if line:  # Ignora linhas vazias
                        current_sequence.append(line)
            
            # Salva a última sequência
            if current_header:
                sequences[current_header] = ''.join(current_sequence)
                
    except FileNotFoundError:
        print(f"Erro: Arquivo '{filename}' não encontrado!")
        sys.exit(1)
    except Exception as e:
        print(f"Erro ao ler arquivo: {e}")
        sys.exit(1)
    
    return sequences

def extract_is_elements(sequences):
    """
    Extrai elementos IS das sequências baseado nos padrões do cabeçalho
    """
    is_elements = {}
    is_types = Counter()
    
    # Padrão para identificar elementos IS no cabeçalho
    # Busca por padrões como "IS21_259", "IS3_176", "ISL3_111", "IS30_241"
    is_pattern = re.compile(r'IS[A-Z]*\d*_\d+')
    
    for header, sequence in sequences.items():
        # Procura por padrões IS no cabeçalho
        is_matches = is_pattern.findall(header)
        
        if is_matches:
            for is_match in is_matches:
                # Extrai o tipo de IS (ex: IS21, IS3, ISL3, IS30)
                is_type_match = re.match(r'(IS[A-Z]*\d*)', is_match)
                if is_type_match:
                    is_type = is_type_match.group(1)
                    is_types[is_type] += 1
                    
                    # Armazena a sequência completa com informações adicionais
                    is_elements[is_match] = {
                        'header': header,
                        'sequence': sequence,
                        'type': is_type,
                        'length': len(sequence)
                    }
    
    return is_elements, is_types

def write_is_sequences(is_elements, output_file):
    """
    Escreve as sequências IS em um arquivo FASTA
    """
    try:
        with open(output_file, 'w') as file:
            for is_id, data in is_elements.items():
                file.write(f">{data['header']}\n")
                # Quebra a sequência em linhas de 80 caracteres
                sequence = data['sequence']
                for i in range(0, len(sequence), 80):
                    file.write(f"{sequence[i:i+80]}\n")
        print(f"Sequências IS salvas em: {output_file}")
    except Exception as e:
        print(f"Erro ao escrever arquivo de saída: {e}")

def print_statistics(is_elements, is_types):
    """
    Imprime estatísticas dos elementos IS encontrados
    """
    print("\n" + "="*60)
    print("RELATÓRIO DE ELEMENTOS IS ENCONTRADOS")
    print("="*60)
    
    print(f"\nTotal de elementos IS encontrados: {len(is_elements)}")
    
    if not is_elements:
        print("Nenhum elemento IS foi encontrado no arquivo.")
        return
    
    print(f"\nTipos de IS encontrados: {len(is_types)}")
    print("\nContagem por tipo de IS:")
    print("-" * 30)
    for is_type, count in sorted(is_types.items()):
        print(f"{is_type:<15}: {count:>5}")
    
    print(f"\nEstatísticas de comprimento:")
    print("-" * 30)
    lengths = [data['length'] for data in is_elements.values()]
    print(f"Comprimento mínimo: {min(lengths):,} bp")
    print(f"Comprimento máximo: {max(lengths):,} bp")
    print(f"Comprimento médio:  {sum(lengths)//len(lengths):,} bp")
    
    print(f"\nDetalhes dos elementos IS:")
    print("-" * 50)
    for is_id, data in sorted(is_elements.items()):
        print(f"{is_id:<15} | {data['type']:<10} | {data['length']:>6,} bp")

def main():
    parser = argparse.ArgumentParser(
        description='Extrai e conta elementos IS de arquivos FASTA',
        formatter_class=argparse.RawDescriptionHelpFormatter,
        epilog="""
Exemplos de uso:
  python3 is_extractor.py Homo_sapiens.fa
  python3 is_extractor.py Homo_sapiens.fa elementos_is.fasta
  python3 is_extractor.py input.fasta output.fasta
        """
    )
    
    parser.add_argument('input_file', help='Arquivo FASTA de entrada')
    parser.add_argument('output_file', nargs='?', 
                       help='Arquivo FASTA de saída (opcional)')
    parser.add_argument('--verbose', '-v', action='store_true',
                       help='Saída mais detalhada')
    
    args = parser.parse_args()
    
    # Define arquivo de saída padrão se não especificado
    if not args.output_file:
        base_name = args.input_file.rsplit('.', 1)[0]
        args.output_file = f"{base_name}_IS_elements.fasta"
    
    print(f"Processando arquivo: {args.input_file}")
    
    # Parse do arquivo FASTA
    sequences = parse_fasta(args.input_file)
    print(f"Total de sequências lidas: {len(sequences)}")
    
    # Extração dos elementos IS
    is_elements, is_types = extract_is_elements(sequences)
    
    # Saída das estatísticas
    print_statistics(is_elements, is_types)

if __name__ == "__main__":
    main()

# Ctrl+O e adicionar o nome do arquivo (is_extractor.py)
# Ctrl+X = para sair

# Utilizando o scritpt 
python is_extractor.py serotypeIII_Bovine.fa

In [ ]:
# Diagrama de veen A - Diagrama de Venn do sorotipo Ia dos 3 hospedeiros ############################################################################

# Primeiro entramos no diretório do serotypeIa
## Agora copiamos o arquivo do ISEScan da cepa do sorotipo Ia para o diretório serotypeIa
### Para isso devemos encontrar o caminho (path) do resultado do ISEScan do respectivo sorotipo 
find path/strain/genoma -iname *is.fna
find /mnt/dados/victor_baca/outputs/isescan/01173/genoma -iname *is.fna

## Agora vamos copiar o arquivo is.fna para o diretório serotypeIa, mudando o nome do arquivo para o nome da strain_host.is.fna
cp [arquivo_origem] [diretório_destino/novo_nome]
cp /mnt/dados/victor_baca/outputs/isescan/01173/genoma/GCF_013000945.1_ASM1300094v1_genomic.fna.is.fna /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIa/01173_Fish.is.fna

### Depois de ter copiado os arquivos de todas as strains do sorotipo Ia, vamos concatenar os arquivos por hospedeiro 
cat /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIa/*_Fish.is.fna > serotypeIa_Fish.fasta # Concatena os arquivos de Fish 
cat /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIa/*_Human.is.fna > serotypeIa_Human.fasta # Concatena os arquivos de Humanos
cat /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIa/*_Bovine.is.fna > serotypeIa_Bovine.fasta # Concatena os arquivos de Bovinos

cat /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIa/*_Fish.is.fna /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/*_Human.is.fna > serotypeIa_Fish_Human.fasta # Concatena os arquivos de Fish e Humanos

cat /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIa/*_Fish.is.fna /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/*_Bovine.is.fna > serotypeIa_Fish_Bovine.fasta # Concatena os arquivos de Fish e Bovinos

cat /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIa/*_Human.is.fna /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/*_Bovine.is.fna > serotypeIa_Human_Bovine.fasta # Concatena os arquivos de Humanos e Bovinos

cat /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIa/*_Fish.is.fna /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/*_Human.is.fna /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/*_Bovine.is.fna > serotypeIa_Fish_Human_Bovine.fasta # Concatena os arquivos de Fish, Humanos e Bovinos

### Por ultimo utilizar o cd-hit para reduzir a redundância dos arquivos/anotações
export PATH=$PATH::/mnt/dados/victor_baca/tools/cdhit # Adiciona o caminho do cd-hit ao PATH

nohup cd-hit-est -i /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIa/CD-HIT/serotypeIa_Bovine.fasta -s 0.9 -g 1 -o serotypeIa_Bovine.fa & # serotypeIa_Bovine.fa, este arquivo vai representar/conter os ISs exclusivos do sorotipo Ia de Bovinos
nohup cd-hit-est -i /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIa/CD-HIT/serotypeIa_Fish.fasta -s 0.9 -g 1 -o serotypeIa_Fish.fa & # serotypeIa_Fish.fa, este arquivo vai representar/conter os ISs exclusivos do sorotipo Ia de Peixes
nohup cd-hit-est -i /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIa/CD-HIT/serotypeIa_Human.fasta -s 0.9 -g 1 -o serotypeIa_Human.fa & # serotypeIa_Human.fa, este arquivo vai representar/conter os exclusivos ISs do sorotipo Ia de Humanos

nohup cd-hit-est -i /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIa/CD-HIT/serotypeIa_Host/serotypeIa_Fish_Human.fasta -s 0.9 -g 1 -o serotypeIa_Fish_Human.fa & # serotypeIa_Fish_Human.fa, este arquivo é a interseção de todos os ISs de 2 hospedeiros (Peixe e Humano) do sorotipo Ia

nohup cd-hit-est -i /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIa/CD-HIT/serotypeIa_Host/serotypeIa_Fish_Bovine.fasta -s 0.9 -g 1 -o serotypeIa_Fish_Bovine.fa & # serotypeIa_Fish_Bovine.fa, este arquivo é a interseção de todos os ISs de 2 hospedeiros (Peixe e Bovino) do sorotipo Ia

nohup cd-hit-est -i /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIa/CD-HIT/serotypeIa_Host/serotypeIa_Human_Bovine.fasta -s 0.9 -g 1 -o serotypeIa_Human_Bovine.fa & # serotypeIa_Human_Bovine.fa, este arquivo é a interseção de todos os ISs de 2 hospedeiros (Humano e Bovino) do sorotipo Ia

nohup cd-hit-est -i /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIa/CD-HIT/serotypeIa_Host/serotypeIa_Fish_Human_Bovine.fasta -s 0.9 -g 1 -o serotypeIa_Fish_Human_Bovine.fa & # serotypeIa_Fish_Human_Bovine.fa, este arquivo é a interseção de todos os ISs dos 3 hospedeiros do sorotipo Ia 

# Agora e so utilizar o script is_extractor.py para quantificar os ISs de cada arquivo 

In [ ]:
# Diagrama de veen B - Diagrama de Venn do sorotipo III dos 3 hospedeiros ###########################################################################

# Primeiro entramos no diretório do serotypeIII
## Agora copiamos o arquivo do ISEScan da cepa do sorotipo III para o diretório serotypeIII
### Para isso devemos encontrar o caminho (path) do resultado do ISEScan do respectivo sorotipo 
find path/strain/genoma -iname *is.fna
find /mnt/dados/victor_baca/outputs/isescan/NEM316/genoma -iname *is.fna

## Agora vamos copiar o arquivo is.fna para o diretório serotypeIII, mudando o nome do arquivo para o nome da strain_host.is.fna
cp [arquivo_origem] [diretório_destino/novo_nome]
cp /mnt/dados/victor_baca/outputs/isescan/NEM316/genoma/GCF_030489725.1_ASM3048972v1_genomic.fna.is.fna /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/NEM316_Human.is.fna

### Depois de ter copiado os arquivos de todas as strains do sorotipo III, vamos concatenar os arquivos por hospedeiro 
cat /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/*_Fish* > serotypeIII_Fish.fasta # Concatena os arquivos de Fish 
cat /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/*_Human* > serotypeIII_Human.fasta # Concatena os arquivos de Humanos
cat /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/*_Bovine* > serotypeIII_Bovine.fasta # Concatena os arquivos de Bovinos

cat /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/*_Fish* /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/*_Human* > serotypeIII_Fish_Human.fasta # Concatena os arquivos de Fish e Humanos

cat /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/*_Fish* /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/*_Bovine* > serotypeIII_Fish_Bovine.fasta # Concatena os arquivos de Fish e Bovinos

cat /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/*_Human* /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/*_Bovine* > serotypeIII_Human_Bovine.fasta # Concatena os arquivos de Humanos e Bovinos

cat /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/*_Fish* /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/*_Human* /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/*_Bovine* > serotypeIII_Fish_Human_Bovine.fasta # Concatena os arquivos de Fish, Humanos e Bovinos

### Por ultimo utilizar o cd-hit para reduzir a redundância dos arquivos/anotações
export PATH=$PATH::/mnt/dados/victor_baca/tools/cdhit # Adiciona o caminho do cd-hit ao PATH

nohup cd-hit-est -i /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/CD-HIT/serotypeIII_Bovine.fasta -s 0.9 -g 1 -o serotypeIII_Bovine.fa & # serotypeIII_Bovine.fa, este arquivo vai representar/conter os ISs exclusivos do sorotipo III de Bovinos
nohup cd-hit-est -i /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/CD-HIT/serotypeIII_Fish.fasta -s 0.9 -g 1 -o serotypeIII_Fish.fa & # serotypeIII_Fish.fa, este arquivo vai representar/conter os ISs exclusivos do sorotipo III de Peixes
nohup cd-hit-est -i /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/CD-HIT/serotypeIII_Human.fasta -s 0.9 -g 1 -o serotypeIII_Human.fa & # serotypeIII_Human.fa, este arquivo vai representar/conter os exclusivos ISs do sorotipo III de Humanos

nohup cd-hit-est -i /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/CD-HIT/serotypeIII_Fish_Human.fasta -s 0.9 -g 1 -o serotypeIII_Fish_Human.fa & # serotypeIII_Fish_Human.fa, este arquivo é a interseção de todos os ISs de 2 hospedeiros (Peixe e Humano) do sorotipo III

nohup cd-hit-est -i /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/CD-HIT/serotypeIII_Fish_Bovine.fasta -s 0.9 -g 1 -o serotypeIII_Fish_Bovine.fa & # serotypeIII_Fish_Bovine.fa, este arquivo é a interseção de todos os ISs de 2 hospedeiros (Peixe e Bovino) do sorotipo III

nohup cd-hit-est -i /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/CD-HIT/serotypeIII_Human_Bovine.fasta -s 0.9 -g 1 -o serotypeIII_Human_Bovine.fa & # serotypeIII_Human_Bovine.fa, este arquivo é a interseção de todos os ISs de 2 hospedeiros (Humano e Bovino) do sorotipo III

nohup cd-hit-est -i /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/CD-HIT/serotypeIII_Fish_Human_Bovine.fasta -s 0.9 -g 1 -o serotypeIII_Fish_Human_Bovine.fa & # serotypeIII_Fish_Human_Bovine.fa, este arquivo é a interseção de todos os ISs dos 3 hospedeiros do sorotipo III 

# Agora e so utilizar o script is_extractor.py para quantificar os ISs de cada arquivo 

In [ ]:
# Diagrama de veen C - Para fazer o diagrama de Venn dos sorotipos (Ia e III) em comum dos 3 hospedeiros (Bovinos, Humanos e Peixes) ################

# Para fazer o diagrama de Venn dos sorotipos (Ia e III) que tem nos 3 hospedeiros (Bovinos, Humanos e Peixes)

# Primeiro vamos copiar os arquivos gerados pelo ISEScan para os respectivos diretórios
## Somente copiar os arquivos que são do sorotipo especificado pelo diretório
### Para isso devemos encontrar o caminho (path) do resultado do ISEScan do respectivo sorotipo 
find path/strain/genoma -iname *is.fna
find /mnt/dados/victor_baca/outputs/isescan/01173/genoma -iname *is.fna

### E agora vamos copiar no diretorio criado para este sorotipo 
cp arquivo_original.fna /path
cp /mnt/dados/victor_baca/outputs/isescan/01173/genoma/GCF_013000945.1_ASM1300094v1_genomic.fna.is.fna /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype/serotypeIII/

### Unir todos esses arquivos em um unico arquivo
cat *.is.fna > serotypeIII.fasta

### Por ultimo utilizar o cd-hit para reduzir a redundância dos arquivos/anotações

export PATH=$PATH::/mnt/dados/victor_baca/tools/cdhit
nohup cd-hit-est -i /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotypeIII/serotypeIII.fasta -s 0.9 -g 1 -o serotypeIII.fa &

#### -s 0.9 significa que as sequências devem ter 90% de identidade para serem consideradas redundantes
#### -g 1 = Ativa o modo "acurado", que é mais lento, mas mais preciso

#####################################################################################################################################################

# Criar um novo diretório para a junção dos arquivos dos dois sorotipos
mkdir serotype_Ia_III

# Copiar os arquivos fasta dos dois sorotipos para o novo diretório
## Primeiro encontrar esses arquivos fasta
find path/serotype -iname *.fasta
find /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotypeIII -iname *.fasta

## E agora copiar os arquivos para o novo diretório
cp path/serotype.fasta path/serotype_Ia_III/
cp /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotypeIII/serotypeIII.fasta /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype_Ia_III/

## Unir os dois arquivos em um único arquivo
cat *.fasta > serotype_Ia_III.fasta

# Por último, utilizar o cd-hit para reduzir a redundância dos arquivos/anotações
export PATH=$PATH::/mnt/dados/victor_baca/tools/cdhit
nohup cd-hit-est -i /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/CD-HIT_IS/serotype_Ia_III/serotype_Ia_III.fasta -s 0.9 -g 1 -o serotype_Ia_III.fa &
#### -s 0.9 significa que as sequências devem ter 90% de identidade para serem consideradas redundantes
#### -g 1 = Ativa o modo "acurado", que é mais lento, mas mais preciso

# Utilizar o script is_extractor.py (script acima) para extrair os ISs dos arquivos serotype*.fa

### Arquivo serotypeIa.fa e serotypeIII.fa no diagrama de Venn serão os conjuntos de serotipos Ia e III respectivamente. Eles vão representar/conter os ISs exclusivos de cada sorotipo. 
### Arquivo serotype_Ia_III.fa no diagrama de veen, sera a interseção entre os serotipos Ia e III